In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv("../input/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [ ]:
data

In [ ]:
# output = (rows, columns)
data.shape 

Which features are numerical?
SeniorCitizen, Tenure, MonthlyCharges, TotalCharges

Continous - Tenure, MonthlyCharges, TotalCharges

Discrete - SeniorCitizen

Which features are categorical?
PhoneService, MultipleLines, InternetService, OnlineSecurity, OnlineBackup DeviceProtection, TechSupport, StreamingTV, StreamingMovies, Contract, PaperlessBilling, PaymentMethod, gender, Partner, Dependents

In [ ]:
data_temp = data.copy()

In [ ]:
data_temp['Churn'].value_counts().plot.pie(autopct='%1.1f%%')

In [ ]:
sns.factorplot(x="Contract", y="MonthlyCharges", hue="PaymentMethod", kind="point", data=data_temp)

In [ ]:
sns.pairplot(data_temp[['tenure','MonthlyCharges','TotalCharges','Churn','Contract','SeniorCitizen']], hue='Churn')

In [ ]:
### shorter the contract period, higher probability of churn? lets check crosstab

cr  = pd.crosstab(data_temp.Contract, data_temp.Churn)
cr.plot(kind='bar')
plt.title('Churn by Contract')
plt.ylabel("Number of chrun")
plt.show()

In [ ]:
data_temp.dtypes

Here we can see that Total Charges is an object variable. Let's Change it to float

In [ ]:
tot_charges = data_temp["TotalCharges"]

In [ ]:
def convert_to_float(strin):
        try:
            return float(strin)
        except:
            return 0

In [ ]:
data_temp['TotalCharges']=data_temp['TotalCharges'].apply(convert_to_float)

In [ ]:
data_temp['TotalCharges']

In [ ]:
#describe the data
data_temp.describe()

In [ ]:
# Checking For NULL 
data_temp.isnull().sum()

In [ ]:
#Identifying the rows containing 0 value in Total Charges
zero_value_row = list(data_temp[data_temp['TotalCharges'] == 0].index)
print('0 Value Rows=',zero_value_row, "\ntotal =", len(zero_value_row))


In [ ]:
for zero_row in zero_value_row :
    print( data_temp['MonthlyCharges'][zero_row],data_temp['tenure'][zero_row],data_temp['TotalCharges'][zero_row])

Interesting ! So Tenure is 0 for the cstomers with TotalCharges as 0, that initially had a space.  might mean that they have not yet been with the company for a whole month, and therefore have not yet been charged any money. This explains why TotalCharges is missing, and also why none of them have churned yet.  At this point we can safely remove these from our model.
 

In [ ]:
# Look at the shape before and after to be sure they were removed
print(data_temp.shape)
data_temp = data_temp.drop(data_temp.index[[488, 753, 936, 1082, 1340, 3331, 3826, 4380, 5218, 6670, 6754]])
print(data_temp.shape)

**Let's Convert Our Target Variable 'Churn' for Yes or No to 1 or 0.**
* Using the logistic regression model, categories must be transformed into numbers first before we can apply the learning algorithm.
* Preprocessing

In [ ]:
#coverting Churn from Yes or No to 1 or 0 and saving it in a new column name class
data_temp['class'] = data_temp['Churn'].apply(lambda x : 1 if x == "Yes" else 0)

In [ ]:
count_no_churn = len(data_temp[data_temp['class']==0])
count_churn = len(data_temp[data_temp['class']==1])
pct_of_no_churn = count_no_churn/(count_no_churn+count_churn)
print("percentage of no churn is", pct_of_no_churn*100)
pct_of_churn = count_churn/(count_no_churn+count_churn)
print("percentage of churn", pct_of_churn*100)

**when churn is "Yes"**

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'MonthlyCharges'].median()

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'TotalCharges'].median()

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'tenure'].median()

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'), 'Contract'].value_counts(normalize = True)

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'PaymentMethod'].value_counts(normalize = True)

**Most of the People that Left are the Ones who had Payment Method as Electronic Check so Let's Make a Seperate Variable for it so that The Model can Easily Predict our Target Variable.**

In [ ]:
data_temp['Is_Electronic_check'] = np.where(data_temp['PaymentMethod'] == 'Electronic check',1,0)

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'PaperlessBilling'].value_counts(normalize = True)

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'), 'Contract'].value_counts(normalize = True)

In [ ]:
data_temp['Is_Contract_Month'] = np.where(data_temp['PaymentMethod'] == 'Month-to-month',1,0)

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'DeviceProtection'].value_counts(normalize = True)

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'OnlineBackup'].value_counts(normalize = True)

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'TechSupport'].value_counts(normalize = True)

In [ ]:
data_temp.loc[(data_temp.Churn == 'Yes'),'OnlineSecurity'].value_counts(normalize = True)

**We can See that People That Left the Company did'nt use Services Like Online Security , Device Protection , Tech Support and Online Backup quite often. Hence for Our Prediction these variables will not be of much Importance. We will Drop them in the End.**

In [ ]:
data_temp= pd.get_dummies(data_temp,columns=['Partner','Dependents',
       'PhoneService', 'MultipleLines','StreamingTV',
       'StreamingMovies','Contract','PaperlessBilling','InternetService'],drop_first=True)

We have Encoded the Categorical Variables with Numeric using get dummies Property which will make it easy for the Machine to Make Correct Prediction.

In [ ]:
data_temp.info()

Now Let's Drop the variables that are not Important For us according to our Analysis.

In [ ]:
data_temp.drop(['StreamingTV_No internet service','StreamingMovies_No internet service'],axis=1,inplace=True)

In [ ]:
data_temp.drop('gender',axis=1,inplace=True)

In [ ]:
data_temp.drop('class',axis=1,inplace=True)

In [ ]:
data_temp.drop('customerID',axis=1,inplace=True)

In [ ]:
#data_temp.drop(['tenure','MonthlyCharges'],axis=1,inplace=True)

In [ ]:
data_temp.drop(['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','PaymentMethod'],axis=1,inplace=True)

In [ ]:
data_temp = pd.get_dummies(data_temp,columns=['Churn'],drop_first=True)

In [ ]:
data_temp.info()

**Let's start with Logistic Regression Model because we know Our Target Variable has a Binary Outcome.**
* Binary logistic regression requires the dependent variable to be binary.
* For a binary regression, the factor level 1 of the dependent variable should represent the desired outcome.
* Only the meaningful variables should be included.

In [ ]:
X = data_temp.drop('Churn_Yes',axis=1).values.astype('float')
y = data_temp['Churn_Yes']

In [ ]:
# train test 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

In [ ]:
# create model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [ ]:
# train model
model.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score,classification_report

In [ ]:
print('Model Score:',model.score(X_train,y_train))

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print ('accuracy for logistic regression - version 1 : {0:.2f}'.format(accuracy_score(y_test, model.predict(X_test))))

In [ ]:
print ('confusion matrix for logistic regression - version 1: \n {0}'.format(confusion_matrix(y_test, model.predict(X_test))))

**The results from the confusion matrix are telling us that 1404 and 265 are the number of correct predictions. 151 and 290 are the number of incorrect predictions.**

Precision is the fraction of correctly identified examples of a class (ratio of true possitive to all positives)

In [ ]:
print ('precision for logistic regression - version 1 : {0:.2f}'.format(precision_score(y_test, model.predict(X_test))))

Recall is the fraction of observation classified in that class that was correctly classified

In [ ]:
print ('recall for logistic regression - version 1 : {0:.2f}'.format(recall_score(y_test, model.predict(X_test))))

In [ ]:
print(classification_report(y_test,model.predict(X_test)))

# Decsion Tree

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data_temp

In [ ]:
data_temp.columns

In [ ]:
X_data = data_temp[['SeniorCitizen','tenure','MonthlyCharges','TotalCharges','Is_Electronic_check','Is_Contract_Month','StreamingTV_Yes']]
y_data = data_temp['Churn_Yes']

In [ ]:
#X_data = data_temp.drop('Churn_Yes',axis=1).values.astype('float')
#y_data = data_temp['Churn_Yes']

In [ ]:

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [ ]:
clf = DecisionTreeClassifier(max_depth=2, criterion='entropy')

In [ ]:
clf.fit(X_data, y_data)

In [ ]:
DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [ ]:
y_data

In [ ]:
pd.DataFrame([ "%.2f%%" % perc for perc in (clf.feature_importances_ * 100) ], index = X_data.columns, columns = ['Feature Significance in Decision Tree'])

In [ ]:
import graphviz

In [ ]:

dot_data = tree.export_graphviz(clf,out_file=None, 
                                feature_names=X_data.columns,
                                class_names = ['No', 'Yes'],
                         filled=True, rounded=True,  proportion=True,
                                node_ids=True, #impurity=False,
                         special_characters=True)

In [ ]:
graph = graphviz.Source(dot_data) 
graph